In [1]:
import numpy as np
import pandas as pd
import folium

from pytrack.graph import graph, distance
from pytrack.analytics import visualization
from pytrack.matching import candidate, mpmatching_utils, mpmatching

In [9]:
def create_plot(data):
    m = folium.Map(location=[-7.758360, 110.395460], zoom_start=25)

    # Create a PolyLine to connect the points
#     locations = data[['lat', 'lng']].values.tolist()
#     folium.PolyLine(
#         locations=locations,
#         color='red',  # Color of the line
#         weight=2,  # Thickness of the line
#         opacity=1,  # Opacity of the line
#     ).add_to(m)

    # Add CircleMarkers for each point
    for index, row in data.iterrows():
        folium.CircleMarker(
            location=[row["latitude"], row["longitude"]],
            radius=5,  # Marker size
            color="blue",  # Marker color
            fill=True,
            fill_color="blue",  # Fill color of the marker
            fill_opacity=0.7,  # Opacity of the marker fill
            popup=f"User ID: {row['maid']}<br>Latitude: {row['latitude']}<br>Longitude: {row['longitude']}",
        ).add_to(m)
    
    return m

In [10]:
def create_pivot(data):
    df1_1 = data.copy()
    df1_pivot = df1_1['maid'].groupby(df1_1['tanggal']).value_counts()
    pivot = df1_pivot.unstack().fillna(0).astype(int)

    total_counts = pivot.sum(axis=0)
    sorted_columns = total_counts.sort_values(ascending=False).index
    pivot_sorted = pivot[sorted_columns]
    return pivot_sorted

In [77]:
def remove_rows_time_range(df, time_range):
    # Convert 'datetime_wib' column to datetime type
    df.loc[:, 'datetime_wib'] = pd.to_datetime(df['datetime_wib'])

    # Sort the DataFrame by the 'datetime_wib' column
    df = df.sort_values('datetime_wib')

    # Calculate the time difference between consecutive rows
    time_diff = df['datetime_wib'].diff()

    # Keep rows where time difference is greater than or equal to 5 seconds
    df_filtered = df[time_diff >= pd.Timedelta(seconds=time_range)]

    return df_filtered

## Mencoba adjust path

In [94]:
data

,maid,latitude,longitude,datetime_wib,geometry,Kelurahan/Desa,Kecamatan,Kabupaten,tanggal
269,0d21360c-6686-4e55-bdc9-7f64e5996255,-7.75838,110.395699,2021-11-08 14:23:03,POINT (110.39569854736328 -7.75838),Condongcatur,Depok,Sleman,2021-11-08
271,0d21360c-6686-4e55-bdc9-7f64e5996255,-7.75814,110.395729,2021-11-08 14:23:09,POINT (110.3957290649414 -7.75814),Condongcatur,Depok,Sleman,2021-11-08
272,0d21360c-6686-4e55-bdc9-7f64e5996255,-7.75874,110.395561,2021-11-08 14:23:36,POINT (110.39556121826172 -7.75874),Condongcatur,Depok,Sleman,2021-11-08
273,0d21360c-6686-4e55-bdc9-7f64e5996255,-7.75876,110.395561,2021-11-08 14:23:46,POINT (110.39556121826172 -7.75876),Condongcatur,Depok,Sleman,2021-11-08
274,0d21360c-6686-4e55-bdc9-7f64e5996255,-7.75859,110.395630,2021-11-08 14:23:50,POINT (110.3956298828125 -7.75859),Condongcatur,Depok,Sleman,2021-11-08
275,0d21360c-6686-4e55-bdc9-7f64e5996255,-7.75837,110.395699,2021-11-08 14:23:54,POINT (110.39569854736328 -7.75837),Condongcatur,Depok,Sleman,2021-11-08
276,0d21360c-6686-4e55-bdc9-7f64e5996255,-7.75816,110.395737,2021-11-08 14:23:59,POINT (110.39573669433594 -7.75816),Condongcatur,Depok,Sleman,2021-11-08
277,0d21360c-6686-4e55-bdc9-7f64e5996255,-7.75790,110.395798,2021-11-08 14:24:02,POINT (110.3957977294922 -7.7579),Condongcatur,Depok,Sleman,2021-11-08
278,0d21360c-6686-4e55-bdc9-7f64e5996255,-7.75790,110.395798,2021-11-08 14:24:51,POINT (110.3957977294922 -7.7579),Condongcatur,Depok,Sleman,2021-11-08


In [95]:
create_plot(data)

## -----------------------------------------------------------------

In [92]:
data = remove_rows_time_range(df1, 3)
data

,maid,latitude,longitude,datetime_wib,geometry,Kelurahan/Desa,Kecamatan,Kabupaten,tanggal
269,0d21360c-6686-4e55-bdc9-7f64e5996255,-7.75838,110.395699,2021-11-08 14:23:03,POINT (110.39569854736328 -7.75838),Condongcatur,Depok,Sleman,2021-11-08
271,0d21360c-6686-4e55-bdc9-7f64e5996255,-7.75814,110.395729,2021-11-08 14:23:09,POINT (110.3957290649414 -7.75814),Condongcatur,Depok,Sleman,2021-11-08
272,0d21360c-6686-4e55-bdc9-7f64e5996255,-7.75874,110.395561,2021-11-08 14:23:36,POINT (110.39556121826172 -7.75874),Condongcatur,Depok,Sleman,2021-11-08
273,0d21360c-6686-4e55-bdc9-7f64e5996255,-7.75876,110.395561,2021-11-08 14:23:46,POINT (110.39556121826172 -7.75876),Condongcatur,Depok,Sleman,2021-11-08
274,0d21360c-6686-4e55-bdc9-7f64e5996255,-7.75859,110.395630,2021-11-08 14:23:50,POINT (110.3956298828125 -7.75859),Condongcatur,Depok,Sleman,2021-11-08
275,0d21360c-6686-4e55-bdc9-7f64e5996255,-7.75837,110.395699,2021-11-08 14:23:54,POINT (110.39569854736328 -7.75837),Condongcatur,Depok,Sleman,2021-11-08
276,0d21360c-6686-4e55-bdc9-7f64e5996255,-7.75816,110.395737,2021-11-08 14:23:59,POINT (110.39573669433594 -7.75816),Condongcatur,Depok,Sleman,2021-11-08
277,0d21360c-6686-4e55-bdc9-7f64e5996255,-7.75790,110.395798,2021-11-08 14:24:02,POINT (110.3957977294922 -7.7579),Condongcatur,Depok,Sleman,2021-11-08
278,0d21360c-6686-4e55-bdc9-7f64e5996255,-7.75790,110.395798,2021-11-08 14:24:51,POINT (110.3957977294922 -7.7579),Condongcatur,Depok,Sleman,2021-11-08


In [79]:
df1

,maid,latitude,longitude,datetime_wib,geometry,Kelurahan/Desa,Kecamatan,Kabupaten,tanggal
49,0d21360c-6686-4e55-bdc9-7f64e5996255,-7.75922,110.395370,2021-11-08 14:19:00,POINT (110.39537048339844 -7.75922),Condongcatur,Depok,Sleman,2021-11-08
50,0d21360c-6686-4e55-bdc9-7f64e5996255,-7.75922,110.395370,2021-11-08 14:19:01,POINT (110.39537048339844 -7.75922),Condongcatur,Depok,Sleman,2021-11-08
51,0d21360c-6686-4e55-bdc9-7f64e5996255,-7.75922,110.395370,2021-11-08 14:19:02,POINT (110.39537048339844 -7.75922),Condongcatur,Depok,Sleman,2021-11-08
52,0d21360c-6686-4e55-bdc9-7f64e5996255,-7.75922,110.395370,2021-11-08 14:19:03,POINT (110.39537048339844 -7.75922),Condongcatur,Depok,Sleman,2021-11-08
53,0d21360c-6686-4e55-bdc9-7f64e5996255,-7.75922,110.395370,2021-11-08 14:19:04,POINT (110.39537048339844 -7.75922),Condongcatur,Depok,Sleman,2021-11-08
...,...,...,...,...,...,...,...,...,...
274,0d21360c-6686-4e55-bdc9-7f64e5996255,-7.75859,110.395630,2021-11-08 14:23:50,POINT (110.3956298828125 -7.75859),Condongcatur,Depok,Sleman,2021-11-08
275,0d21360c-6686-4e55-bdc9-7f64e5996255,-7.75837,110.395699,2021-11-08 14:23:54,POINT (110.39569854736328 -7.75837),Condongcatur,Depok,Sleman,2021-11-08
276,0d21360c-6686-4e55-bdc9-7f64e5996255,-7.75816,110.395737,2021-11-08 14:23:59,POINT (110.39573669433594 -7.75816),Condongcatur,Depok,Sleman,2021-11-08
277,0d21360c-6686-4e55-bdc9-7f64e5996255,-7.75790,110.395798,2021-11-08 14:24:02,POINT (110.3957977294922 -7.7579),Condongcatur,Depok,Sleman,2021-11-08


In [71]:
path = '../../DataGPS_Affandi/filter3_affandi_nov.csv'
df = pd.read_csv(path)
df.head()

,maid,latitude,longitude,datetime_wib,geometry,Kelurahan/Desa,Kecamatan,Kabupaten,tanggal
0,005000b0-954c-40df-9ad1-d9bf8bd4187a,-7.758964,110.395430,2021-11-20 04:50:11,POINT (110.39543 -7.758964),Condongcatur,Depok,Sleman,2021-11-20
1,005000b0-954c-40df-9ad1-d9bf8bd4187a,-7.758964,110.395432,2021-11-20 04:50:13,POINT (110.39543151855467 -7.758964),Condongcatur,Depok,Sleman,2021-11-20
2,005000b0-954c-40df-9ad1-d9bf8bd4187a,-7.758964,110.395430,2021-11-20 04:50:14,POINT (110.39543 -7.758964),Condongcatur,Depok,Sleman,2021-11-20
3,005000b0-954c-40df-9ad1-d9bf8bd4187a,-7.758970,110.395430,2021-11-20 04:50:15,POINT (110.39543 -7.75897),Condongcatur,Depok,Sleman,2021-11-20
4,005000b0-954c-40df-9ad1-d9bf8bd4187a,-7.758960,110.395439,2021-11-20 04:50:19,POINT (110.39543914794922 -7.75896),Condongcatur,Depok,Sleman,2021-11-20


In [16]:
create_pivot(df)

maid,0d21360c-6686-4e55-bdc9-7f64e5996255,8516b4c0-d7df-4f73-ad5a-f21039f32d0c,27683c4b-5c14-4a14-9788-9aba0a7b0b38,9dae5a26-7a6a-4aae-852c-c201cee301f0,dfe1c884-187c-4269-91ad-cd43c503a814,22648087-8a28-436a-9b66-6d97ab42fa0c,a10cf86d-4225-4f86-8cf3-630d07895c24,deb74559-fb6d-4493-8000-da0c8d1758df,4213fc32-db1f-4322-be8e-e4230784254d,ebf04e91-9f51-44c7-922d-6437603bc8dc,...,1b23baf7-72f0-45e6-8102-ae76f7e4130a,ee873a66-1892-4b1a-a2d9-aeadd11daddc,af62b9ce-3ff9-4f78-a36f-0d9546089dde,40090678-0147-4edf-b266-e2acd552f502,24ceefee-8ba4-457c-97c7-79fcb69cc14f,c3c3a1b8-e981-494d-aa91-c1afb933296e,6eb63b0a-06f8-472d-a219-4d2a9f2ce12f,2f36ab56-dd41-4633-aab8-b56be0164c1d,30d02d2e-ebc9-4766-8ab8-36459a8a3924,7a294c1b-ce55-4371-9bf3-7c9fc1d0e2aa
tanggal,,,,,,,,,,,,,,,,,,,,,
2021-11-01,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2021-11-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-11-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-11-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-11-05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-11-06,0,0,0,26,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-11-07,0,0,23,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2021-11-08,230,0,0,0,0,0,22,22,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-11-09,0,0,0,0,0,0,0,0,21,0,...,0,0,0,0,0,0,0,0,0,0


In [93]:
df1 = df[(df['maid'] == '0d21360c-6686-4e55-bdc9-7f64e5996255') & (df['tanggal'] == '2021-11-08')]
create_plot(data)

In [83]:
df1 = data

In [84]:
latitude = df1["latitude"].to_list()
longitude = df1["longitude"].to_list()

points = [(lat, lon) for lat, lon in zip(latitude, longitude)]

In [85]:
north, east = np.max(np.array([*points]), 0)
south, west = np.min(np.array([*points]), 0)
G = graph.graph_from_bbox(*distance.enlarge_bbox(north, south, west, east, 200), simplify=True, network_type='drive')

Downloaded 78.03kB


In [86]:
# Initialize maps
loc = (np.mean(latitude[:30]), np.mean(longitude[:30]))
maps = visualization.Map(location=loc, zoom_start=15)
#maps

In [87]:
# Show extracted graph
maps.add_graph(G, plot_nodes=True)

In [88]:
# Extract candidates
G_interp, candidates = candidate.get_candidates(G, points, interp_dist=5, closest=True, radius=30)

# Plot results
maps.draw_candidates(candidates, 30)
maps

In [124]:
# c = candidate.elab_candidate_results(candidates, predecessor)
# c = pd.DataFrame(c)
# c

In [89]:
# Extract trellis DAG graph
trellis = mpmatching_utils.create_trellis(candidates)

# Plot trellis graph
# trellis_draw = visualization.draw_trellis(trellis, figsize=(15, 100), dpi=200)
# trellis_draw

In [90]:
# Perform the map-matching process
path_prob, predecessor = mpmatching.viterbi_search(G_interp, trellis, "start", "target")

Node 461616726 not reachable from 509977346
Node 513608126 not reachable from 509977346
Node 1269509566 not reachable from 509977346
Node 435554224 not reachable from 509977346
Node 509977346 not reachable from 2075140017
Node 461616726 not reachable from 2075140017
Node 513608126 not reachable from 2075140017
Node 1269509566 not reachable from 2075140017
Node 435554224 not reachable from 2075140017
Node 494829805 not reachable from 509977346
Node 664961377 not reachable from 509977346
Node 102790558 not reachable from 509977346
Node 777449925 not reachable from 509977346
Node 2118423424 not reachable from 509977346
Node 494829805 not reachable from 509977346
Node 664961377 not reachable from 509977346
Node 1902320653 not reachable from 509977346
Node 715459257 not reachable from 509977346
Node 1020442352 not reachable from 509977346
Node 435554224 not reachable from 509977346
Node 715459257 not reachable from 509977346
Node 853780466 not reachable from 1807984268
Node 715459257 not re

In [91]:
# Plot map-matching results
maps.draw_path(G_interp, trellis, predecessor)
maps

In [129]:
candidate

<module 'pytrack.matching.candidate' from '/raid/arkham/miniconda3/envs/arkham/lib/python3.10/site-packages/pytrack/matching/candidate.py'>

In [144]:
matched_path = mpmatching_utils.create_matched_path(G_interp, trellis, predecessor)
matched_path = matched_path[1]
matched_path_df = pd.DataFrame(matched_path)

In [145]:
matched_path_df

,0,1
0,-7.790281,110.366885
1,-7.790279,110.366845
2,-7.790278,110.366805
3,-7.790230,110.366808
4,-7.790182,110.366813
...,...,...
152,-7.796013,110.365453
153,-7.796057,110.365448
154,-7.796102,110.365443
155,-7.796146,110.365438


In [131]:
df1

,maid,latitude,longitude,datetime_wib,geometry,Kelurahan/Desa,Kecamatan,Kabupaten,tanggal
3016,f501c8a1-e251-4b8c-87fc-9a1a26d2e19d,-7.790250,110.366867,2021-12-14 14:45:45,POINT (110.36686706542967 -7.79025),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta,2021-12-14
3017,f501c8a1-e251-4b8c-87fc-9a1a26d2e19d,-7.790060,110.366410,2021-12-14 14:46:32,POINT (110.36641 -7.79006),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta,2021-12-14
3018,f501c8a1-e251-4b8c-87fc-9a1a26d2e19d,-7.790060,110.366410,2021-12-14 14:46:41,POINT (110.36641 -7.79006),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta,2021-12-14
3019,f501c8a1-e251-4b8c-87fc-9a1a26d2e19d,-7.790110,110.366158,2021-12-14 14:47:34,POINT (110.36615753173828 -7.79011),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta,2021-12-14
3020,f501c8a1-e251-4b8c-87fc-9a1a26d2e19d,-7.790110,110.366160,2021-12-14 14:47:43,POINT (110.36616 -7.79011),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta,2021-12-14
3021,f501c8a1-e251-4b8c-87fc-9a1a26d2e19d,-7.791284,110.366020,2021-12-14 14:48:35,POINT (110.36602 -7.7912836),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta,2021-12-14
3022,f501c8a1-e251-4b8c-87fc-9a1a26d2e19d,-7.791270,110.366013,2021-12-14 14:48:40,POINT (110.3660125732422 -7.79127),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta,2021-12-14
3023,f501c8a1-e251-4b8c-87fc-9a1a26d2e19d,-7.792460,110.366100,2021-12-14 14:49:35,POINT (110.3661 -7.79246),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta,2021-12-14
3024,f501c8a1-e251-4b8c-87fc-9a1a26d2e19d,-7.792450,110.366096,2021-12-14 14:49:40,POINT (110.36609649658205 -7.79245),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta,2021-12-14
3025,f501c8a1-e251-4b8c-87fc-9a1a26d2e19d,-7.796204,110.365471,2021-12-14 14:50:37,POINT (110.36547088623048 -7.796204),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta,2021-12-14


In [132]:
def create_plot_path(data):
    m = folium.Map(location=data[:1][:1], zoom_start=25)

    # Add CircleMarkers for each point
    for index, row in data.iterrows():
        folium.CircleMarker(
            location=(row[0], row[1]),
            radius=5,  # Marker size
            color="blue",  # Marker color
            fill=True,
            fill_color="blue",  # Fill color of the marker
            fill_opacity=0.7,  # Opacity of the marker fill
        ).add_to(m)
    
    return m

In [133]:
from math import radians, cos, sin, asin, sqrt

def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

In [134]:
from datetime import datetime

def linear_interpolation(point1, point2, timestamp1, timestamp2, target_point):
    """
    Linear interpolation function to estimate the timestamp of a target point
    given two GPS points and their timestamps.

    Parameters:
        point1 (tuple): Latitude and longitude of the first GPS point (lat1, lon1).
        point2 (tuple): Latitude and longitude of the second GPS point (lat2, lon2).
        timestamp1 (float): Timestamp of the first GPS point.
        timestamp2 (float): Timestamp of the second GPS point.
        target_point (tuple): Latitude and longitude of the target GPS point (lat_target, lon_target).

    Returns:
        float: Estimated timestamp of the target GPS point.
    """

    lat1, lon1 = point1
    lat2, lon2 = point2
    lat_target, lon_target = target_point

    # Convert timestamp strings to datetime objects
    dt1 = datetime.strptime(timestamp1, "%Y-%m-%d %H:%M:%S")
    dt2 = datetime.strptime(timestamp2, "%Y-%m-%d %H:%M:%S")

    # Calculate the distances between points
    dist1 = haversine(lat1, lon1, lat_target, lon_target)
    dist2 = haversine(lat2, lon2, lat_target, lon_target)
    total_dist = haversine(lat1, lon1, lat2, lon2)
    
    # Calculate time difference in seconds
    time_diff = (dt2 - dt1).total_seconds()

    # Calculate the proportional time difference
    interpolated_time_seconds = dt1.timestamp() + (time_diff * dist1 / total_dist)

    # Convert the interpolated timestamp back to string format
    interpolated_timestamp = datetime.fromtimestamp(interpolated_time_seconds).strftime("%Y-%m-%d %H:%M:%S")

    return interpolated_timestamp

# Example usage:
point1 = (-7.790060, 110.366410)  # Latitude and longitude of point 1
point2 = (-7.791270, 110.366013) # Latitude and longitude of point 2
timestamp1 = "2021-12-14 14:46:41" # Timestamp of point 1
timestamp2 = "2021-12-14 14:48:40" # Timestamp of point 2
target_point = (-7.790110, 110.366160)  # Latitude and longitude of the target point (San Francisco)

estimated_timestamp = linear_interpolation(point1, point2, timestamp1, timestamp2, target_point)
print("Estimated timestamp of the target point:", estimated_timestamp)

Estimated timestamp of the target point: 2021-12-14 14:47:04


In [135]:
from datetime import datetime

def calculate_time_difference(timestamp1, timestamp2):
    """
    Calculate the time difference between two timestamps.

    Parameters:
        timestamp1 (str): First timestamp in "YYYY-MM-DD HH:MM:SS" format.
        timestamp2 (str): Second timestamp in "YYYY-MM-DD HH:MM:SS" format.

    Returns:
        int: Time difference in seconds.
    """

    # Convert timestamp strings to datetime objects
    dt1 = datetime.strptime(timestamp1, "%Y-%m-%d %H:%M:%S")
    dt2 = datetime.strptime(timestamp2, "%Y-%m-%d %H:%M:%S")

    # Calculate the time difference in seconds
    time_diff_seconds = abs((dt2 - dt1).total_seconds())

    return time_diff_seconds

In [136]:
def count_vi(lat1, lon1, lat2, lon2, timestamp1, timestamp2):
    di = haversine(lat1, lon1, lat2, lon2)*1000
    time = calculate_time_difference(timestamp1, timestamp2)
    
    vi = di/time
    vi_kph = vi*3.6
    
    return vi_kph

In [137]:
count_vi(-7.790110, 110.366160, -7.796210, 110.365448, "2021-12-14 14:47:34", "2021-12-14 14:50:47")

12.736342889060714

In [138]:
estimated_distance = haversine(-7.790110, 110.366160, -7.801219, 110.364795)

In [140]:
a = pd.DataFrame()
b = matched_path_df.copy()
for index, row in b.iterrows():
    if row[0] <= -7.7901 and row[1] >= 110.3662:
        a = a._append(row)
        print(row[0], row[1])

b.drop(a.index, inplace=True)
b

-7.7902807 110.3668854
-7.79027915 110.3668451
-7.7902776 110.3668048
-7.790229941600696 110.366808267066
-7.79018244717207 110.36681289828691
-7.7901357 110.3668228
-7.7901327636462225 110.3667774583766
-7.790129827292445 110.36673211675323
-7.790126890938667 110.36668677512984
-7.790123954584889 110.36664143350646
-7.790121018231111 110.36659609188307
-7.790118081877333 110.36655075025968
-7.790115145523555 110.3665054086363
-7.790112209169777 110.36646006701291
-7.790109272815999 110.36641472538953
-7.790106336462221 110.36636938376614
-7.790105056337511 110.36632397196169
-7.790104146439665 110.36627854446922
-7.790103236541819 110.36623311697677


,0,1
19,-7.790116,110.366190
20,-7.790155,110.366170
21,-7.790199,110.366165
22,-7.790243,110.366160
23,-7.790288,110.366154
...,...,...
152,-7.796013,110.365453
153,-7.796057,110.365448
154,-7.796102,110.365443
155,-7.796146,110.365438


In [141]:
create_plot_path(b)

In [142]:
matched_path_df

,0,1
0,-7.790281,110.366885
1,-7.790279,110.366845
2,-7.790278,110.366805
3,-7.790230,110.366808
4,-7.790182,110.366813
...,...,...
152,-7.796013,110.365453
153,-7.796057,110.365448
154,-7.796102,110.365443
155,-7.796146,110.365438


In [143]:
df1

,maid,latitude,longitude,datetime_wib,geometry,Kelurahan/Desa,Kecamatan,Kabupaten,tanggal
3016,f501c8a1-e251-4b8c-87fc-9a1a26d2e19d,-7.790250,110.366867,2021-12-14 14:45:45,POINT (110.36686706542967 -7.79025),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta,2021-12-14
3017,f501c8a1-e251-4b8c-87fc-9a1a26d2e19d,-7.790060,110.366410,2021-12-14 14:46:32,POINT (110.36641 -7.79006),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta,2021-12-14
3018,f501c8a1-e251-4b8c-87fc-9a1a26d2e19d,-7.790060,110.366410,2021-12-14 14:46:41,POINT (110.36641 -7.79006),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta,2021-12-14
3019,f501c8a1-e251-4b8c-87fc-9a1a26d2e19d,-7.790110,110.366158,2021-12-14 14:47:34,POINT (110.36615753173828 -7.79011),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta,2021-12-14
3020,f501c8a1-e251-4b8c-87fc-9a1a26d2e19d,-7.790110,110.366160,2021-12-14 14:47:43,POINT (110.36616 -7.79011),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta,2021-12-14
3021,f501c8a1-e251-4b8c-87fc-9a1a26d2e19d,-7.791284,110.366020,2021-12-14 14:48:35,POINT (110.36602 -7.7912836),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta,2021-12-14
3022,f501c8a1-e251-4b8c-87fc-9a1a26d2e19d,-7.791270,110.366013,2021-12-14 14:48:40,POINT (110.3660125732422 -7.79127),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta,2021-12-14
3023,f501c8a1-e251-4b8c-87fc-9a1a26d2e19d,-7.792460,110.366100,2021-12-14 14:49:35,POINT (110.3661 -7.79246),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta,2021-12-14
3024,f501c8a1-e251-4b8c-87fc-9a1a26d2e19d,-7.792450,110.366096,2021-12-14 14:49:40,POINT (110.36609649658205 -7.79245),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta,2021-12-14
3025,f501c8a1-e251-4b8c-87fc-9a1a26d2e19d,-7.796204,110.365471,2021-12-14 14:50:37,POINT (110.36547088623048 -7.796204),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta,2021-12-14
